# Technical Indicators for Financial Analysis

In this work, technical indicators for financial analysis are created to be able to use them in a trading bot. 

**See also** Binance Trader Bot with Python: https://github.com/DrFarukAydin/data-science-portfolio/blob/main/financial-tools/binance-bot.ipynb

**See also** Gate.io Trader Bot with Python: https://github.com/DrFarukAydin/data-science-portfolio/blob/main/financial-tools/gate-io-bot.ipynb

**See also** Data Visualization for Financial Analysis with Python (Jupyter): https://github.com/DrFarukAydin/data-science-portfolio/blob/main/financial-tools/data-visualization-finance.ipynb

<br><br>

**WARNING!!**

<li>The strategies used in this section are <b>not investment advices</b>. They are shown for only educational purposes.
<li>The bot makes real transactions. Therefore, be sure to understand them clearly before implementation. Otherwise, you can lose your money.

<br><br>

## 1. Importing Libraries and Data

<br>**Importing Libraries**

In [65]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import sqlalchemy
import time
from datetime import datetime
import os
import asyncio
import warnings
warnings.filterwarnings('ignore')
from mplfinance.original_flavor import candlestick_ohlc
import pandas_ta as pta

In [2]:
#pip install python-binance

In [3]:
from binance import Client
from binance import BinanceSocketManager

**Note** that some libraries should be installed with pip install.

**Warning:** You should pip install <b>python-binance</b> to be able to import binance

<br><br><br>

## 2. Technical Indicators

### 2.1. SuperTrend

<br>**Definition**

**The Supertrend indicator** is a trend indicator that can be used to determine whether the price is trending up or down. When the price is above the line, it acts as a support point.

<img src='supertrend.png'/>

<br><br>**Parameters & Code**

<ul>
<li><strong>high:&nbsp;</strong>high point of the candlestick</li>
<li><strong>low</strong>: low point of the candlestick</li>
<li><strong>close</strong>: close point of the candlestick</li>
<li><strong>lookback</strong>: how many recent candlesticks for calculation (default "7")</li>
<li><strong>multiplier</strong>: multiplier of ATR value, i.e., width of supertrend line (default "2")&nbsp;</li>
</ul>

In [9]:
def get_supertrend(high, low, close, lookback=7, multiplier=2):
    
    # ATR
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.ewm(lookback).mean()
    
    # H/L AVG AND BASIC UPPER & LOWER BAND
    
    hl_avg = (high + low) / 2
    upper_band = (hl_avg + multiplier * atr).dropna()
    lower_band = (hl_avg - multiplier * atr).dropna()
    
    # FINAL UPPER BAND
    final_bands = pd.DataFrame(columns = ['upper', 'lower'])
    final_bands.iloc[:,0] = [x for x in upper_band - upper_band]
    final_bands.iloc[:,1] = final_bands.iloc[:,0]
    for i in range(len(final_bands)):
        if i == 0:
            final_bands.iloc[i,0] = 0
        else:
            if (upper_band[i] < final_bands.iloc[i-1,0]) | (close[i-1] > final_bands.iloc[i-1,0]):
                final_bands.iloc[i,0] = upper_band[i]
            else:
                final_bands.iloc[i,0] = final_bands.iloc[i-1,0]
    
    # FINAL LOWER BAND
    
    for i in range(len(final_bands)):
        if i == 0:
            final_bands.iloc[i, 1] = 0
        else:
            if (lower_band[i] > final_bands.iloc[i-1,1]) | (close[i-1] < final_bands.iloc[i-1,1]):
                final_bands.iloc[i,1] = lower_band[i]
            else:
                final_bands.iloc[i,1] = final_bands.iloc[i-1,1]
    
    # SUPERTREND
    
    supertrend = pd.DataFrame(columns = [f'supertrend_{lookback}'])
    supertrend.iloc[:,0] = [x for x in final_bands['upper'] - final_bands['upper']]
    
    for i in range(len(supertrend)):
        if i == 0:
            supertrend.iloc[i, 0] = 0
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 0] and close[i] < final_bands.iloc[i, 0]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 0]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 0] and close[i] > final_bands.iloc[i, 0]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 1]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 1] and close[i] > final_bands.iloc[i, 1]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 1]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 1] and close[i] < final_bands.iloc[i, 1]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 0]
    
    supertrend = supertrend.set_index(upper_band.index)
    supertrend = supertrend.dropna()[1:]
    
    # ST UPTREND/DOWNTREND
    
    upt = []
    dt = []
    close = close.iloc[len(close) - len(supertrend):]

    for i in range(len(supertrend)):
        if close[i] > supertrend.iloc[i, 0]:
            upt.append(supertrend.iloc[i, 0])
            dt.append(np.nan)
        elif close[i] < supertrend.iloc[i, 0]:
            upt.append(np.nan)
            dt.append(supertrend.iloc[i, 0])
        else:
            upt.append(np.nan)
            dt.append(np.nan)
            
    st, upt, dt = pd.Series(supertrend.iloc[:, 0]), pd.Series(upt), pd.Series(dt)
    upt.index, dt.index = supertrend.index, supertrend.index
    
    return st, upt, dt

Credit: Nikhil Adithyan @ Medium

<br><br><br>

### 2.2. MOST (Moving Stop Loss)

<br>**Definition**

**The MOST Indicator** is similar to the Supertrend. The difference is calculation technik. The detailed information can be found on the creator's personal page: http://www.teknikanalizsanati.com/most.aspx

<img src='most.png'/>

<br><br>**Parameters & Code**

<ul>
<li><strong>df:&nbsp;</strong>dataframe incl. information about candlestick</li>
<li><strong>lookback</strong>: how many recent candlesticks for calculation (default "8")</li>
<li><strong>percent</strong>: percent value of distance to the top/bottom, i.e., width of most line (default "0.02")&nbsp;</li>
</ul>

In [4]:
def most(df, lookback=8, percent = 0.02):

    df["signal"] = "red"
    df ["greenline"] = df["Close"].rolling(window=lookback).mean()
    df["redline"] = df["greenline"]
    ort_localmax = df.iloc[7]["greenline"]
    ort_localmin = df.iloc[7]["greenline"]
    signal = "*"
    perc = percent
    redline= df.iloc[7]["greenline"]
    for i in range(7,len(df)):
        if ort_localmax < df.iloc[i]["greenline"]:
            ort_localmax = df.iloc[i]["greenline"]
        if ort_localmin > df.iloc[i]["greenline"]:
            ort_localmin = df.iloc[i]["greenline"]

        if df.iloc[i]["greenline"] <= ort_localmax*(1-perc):
            signal = "red"
            redline = ort_localmin*(1+perc)
            if df.iloc[i]["greenline"] > redline:
                signal = "green"

        elif df.iloc[i]["greenline"] > ort_localmin*(1+perc):
            signal = "green"
            redline = ort_localmax*(1-perc)
            if df.iloc[i]["greenline"] < redline:
                signal = "red"


        else:
            signal = df.loc[df.index[i-1],"signal"]

        if signal == "red":
            ort_localmax = df.iloc[i]["greenline"]
            redline = ort_localmin*(1+perc)
        if signal == "green":
            ort_localmin = df.iloc[i]["greenline"]
            redline = ort_localmax*(1-perc)



        df.loc[df.index[i],"signal"] = signal
        df.loc[df.index[i], "slope_greenline"] = (df.iloc[i]["greenline"]-df.iloc[i-1]["greenline"])/df.iloc[i-1]["greenline"]*100
        df.loc[df.index[i],"redline"] = redline



    #needle length
    df["needle_length"] = (df["High"]-df["Low"])/df["Low"]*100
    #body length
    df["body_length"] = (df["Close"]-df["Open"])/df["Open"]*100
    #distance of high to the greenline
    df["high_togreen"] = (df["High"] - df["greenline"])/df["greenline"]*100
    #distance of low to the greenline
    df["low_togreen"] = (df["Low"] - df["greenline"])/df["greenline"]*100
    #distance of high to the redline
    df["high_tored"] = (df["High"] - df["redline"])/df["redline"]*100
    #distance of low to the redline
    df["low_tored"] = (df["Low"] - df["redline"])/df["redline"]*100

    
    return df

Credit: Dr. Ömer Faruk Aydın (The Author of this work)

<br><br><br>

### 2.3. RSI

<br>**Definition**

The relative strength index (RSI) is a technical indicator used in the analysis of financial markets. It is intended to chart the current and historical strength or weakness of a stock or market based on the closing prices of a recent trading period. Source: <a href="https://en.wikipedia.org/wiki/Relative_strength_index">wiki</a>

<img src='rsi.png'/>

<br><br>**Parameters & Code**

<ul>
<li><strong>close:&nbsp;</strong>close price</li>
<li><strong>lookback</strong>: how many recent candlesticks for calculation (default "14")</li>
</ul>

In [3]:
def get_rsi(close, lookback=14):
    ret = close.diff()
    up = []
    down = []
    for i in range(len(ret)):
        if ret[i] < 0:
            up.append(0)
            down.append(ret[i])
        else:
            up.append(ret[i])
            down.append(0)
    up_series = pd.Series(up)
    down_series = pd.Series(down).abs()
    up_ewm = up_series.ewm(com = lookback - 1, adjust = False).mean()
    down_ewm = down_series.ewm(com = lookback - 1, adjust = False).mean()
    rs = up_ewm/down_ewm
    rsi = 100 - (100 / (1 + rs))
    rsi_df = pd.DataFrame(rsi).rename(columns = {0:'rsi'}).set_index(close.index)
    rsi_df = rsi_df.dropna()
    return rsi_df[3:]

Credit: Nikhil Adithyan @ Medium

<br><br><br>

# 3. Combination of Indicators

In this section, a dataframe is created that contains some indicators together.

<br><br>**Parameters & Code**

<ul>
<li><strong>close:&nbsp;</strong>close price</li>
<li><strong>lookback</strong>: how many recent candlesticks for calculation (default "14")</li>
</ul>

In [5]:
def maketable(currency,interval,timeago):
    df = klines_table(currency,interval,timeago)
    df["signal"] = "red"
    df ["greenline"] = df["Close"].rolling(window=8).mean()
    df["redline"] = df["greenline"]
    ort_localmax = df.iloc[7]["greenline"]
    ort_localmin = df.iloc[7]["greenline"]
    signal = "*"
    perc = 0.01
    redline= df.iloc[7]["greenline"]
    for i in range(7,len(df)):
        if ort_localmax < df.iloc[i]["greenline"]:
            ort_localmax = df.iloc[i]["greenline"]
        if ort_localmin > df.iloc[i]["greenline"]:
            ort_localmin = df.iloc[i]["greenline"]

        if df.iloc[i]["greenline"] <= ort_localmax*(1-perc):
            signal = "red"
            redline = ort_localmin*(1+perc)
            if df.iloc[i]["greenline"] > redline:
                signal = "green"

        elif df.iloc[i]["greenline"] > ort_localmin*(1+perc):
            signal = "green"
            redline = ort_localmax*(1-perc)
            if df.iloc[i]["greenline"] < redline:
                signal = "red"


        else:
            signal = df.loc[df.index[i-1],"signal"]

        if signal == "red":
            ort_localmax = df.iloc[i]["greenline"]
            redline = ort_localmin*(1+perc)
        if signal == "green":
            ort_localmin = df.iloc[i]["greenline"]
            redline = ort_localmax*(1-perc)



        df.loc[df.index[i],"signal"] = signal
        df.loc[df.index[i], "slope_greenline"] = (df.iloc[i]["greenline"]-df.iloc[i-1]["greenline"])/df.iloc[i-1]["greenline"]*100
        df.loc[df.index[i],"redline"] = redline



    #needle length
    df["needle_length"] = (df["High"]-df["Low"])/df["Low"]*100
    #body length
    df["body_length"] = (df["Close"]-df["Open"])/df["Open"]*100
    #distance of high to the greenline
    df["high_togreen"] = (df["High"] - df["greenline"])/df["greenline"]*100
    #distance of low to the greenline
    df["low_togreen"] = (df["Low"] - df["greenline"])/df["greenline"]*100
    #distance of high to the redline
    df["high_tored"] = (df["High"] - df["redline"])/df["redline"]*100
    #distance of low to the redline
    df["low_tored"] = (df["Low"] - df["redline"])/df["redline"]*100

    #rsi
    df["rsi"] = get_rsi(df['Close'], 14)
    #df["signal_dummy"]=pd.get_dummies(df["signal"])["green"]
    
    st, upt, dt = get_supertrend(df["High"], df["Low"],df["Close"])
    df = pd.concat([df, st, upt, dt], axis = 1)
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Close_time', 'volume',
       'Number_of_trades', 'Taker_buy_base_asset_vol',
       'Taker_buy_quote_asset_vol', 'Ignore', 'signal', 'greenline', 'redline',
       'slope_greenline', 'needle_length', 'body_length', 'high_togreen',
       'low_togreen', 'high_tored', 'low_tored',"rsi", 'supertrend', 'upt', 'dt']
    df["st_signal"] = np.where(df["upt"] > 0, 1, 0)
    
    return df

The result will be as follows:

In [32]:
maketable("BTCUSDT","15m","300m")

,Open,High,Low,Close,Volume,Close_time,volume,Number_of_trades,Taker_buy_base_asset_vol,Taker_buy_quote_asset_vol,...,body_length,high_togreen,low_togreen,high_tored,low_tored,rsi,supertrend,upt,dt,st_signal
Time,,,,,,,,,,,,,,,,,,,,,
2022-07-14 14:00:00,19765.35,19776.22,19671.91,19749.34,1730.63652,1.657808e+12,3.412316e+07,52768.0,854.89941,1.685700e+07,...,-0.081000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-07-14 14:15:00,19751.60,19766.82,19692.08,19744.18,1351.61716,1.657809e+12,2.667453e+07,41936.0,627.90459,1.239249e+07,...,-0.037567,NaN,NaN,NaN,NaN,NaN,19906.528667,NaN,19906.528667,0
2022-07-14 14:30:00,19746.57,19816.75,19694.20,19782.33,1695.65164,1.657810e+12,3.351133e+07,48817.0,892.43213,1.763959e+07,...,0.181095,NaN,NaN,NaN,NaN,NaN,19906.528667,NaN,19906.528667,0
2022-07-14 14:45:00,19782.25,19957.35,19771.61,19893.58,2816.65665,1.657811e+12,5.597863e+07,66509.0,1459.11023,2.900069e+07,...,0.562777,NaN,NaN,NaN,NaN,NaN,19906.528667,NaN,19906.528667,0
2022-07-14 15:00:00,19895.57,20102.12,19893.46,20037.86,3493.19197,1.657812e+12,6.989040e+07,82080.0,1832.23047,3.666138e+07,...,0.715184,NaN,NaN,NaN,NaN,98.548403,19702.042216,19702.042216,NaN,1
2022-07-14 15:15:00,20037.86,20198.00,20025.00,20142.66,3786.60002,1.657813e+12,7.629710e+07,79830.0,1902.51277,3.833322e+07,...,0.523010,NaN,NaN,NaN,NaN,98.960582,19804.356219,19804.356219,NaN,1
2022-07-14 15:30:00,20142.66,20389.00,20063.60,20318.34,4816.47208,1.657813e+12,9.740633e+07,90824.0,2326.12435,4.705064e+07,...,0.872179,NaN,NaN,NaN,NaN,99.312831,19848.422253,19848.422253,NaN,1
2022-07-14 15:45:00,20320.33,20466.00,20288.00,20390.50,5083.24862,1.657814e+12,1.036076e+08,98799.0,2616.55506,5.334034e+07,...,0.345319,2.292414,1.402741,1.279618,0.398753,99.402413,20003.288547,20003.288547,NaN,1
2022-07-14 16:00:00,20390.00,20485.92,20267.45,20327.48,3864.56128,1.657815e+12,7.874131e+07,81271.0,1828.08247,3.724939e+07,...,-0.306621,2.023464,0.935445,1.378195,0.297058,88.545883,20003.288547,20003.288547,NaN,1
